# AI Learning 07 - 2. MLP training(1)

## MLP

### Data Preprocessing

가장 먼저 고려해야할 점은 feature 별로 봤을 때, 숫자의 범위가 굉장히 제각각이다.<br>
feature 별로 scale을 normalize하는 작업이 필요하다.<br>
이 작업이 있을 때와 없을 때의 성능 차이도 있고, 있을 때 학습이 더 잘 되는 경향이 있다.

In [66]:
# feature 30개를 가져와서 작업한다.
# X 값은 'SalePrice'를 제외한 feature 30개를 가져옴
# y 값은 'SalePrice'를 가져옴
X = df_processed[df_features[-31:-1].index.tolist()]
y = df_processed['SalePrice']

In [67]:
X

,MoSold,LandSlope_rank,SaleType_rank,LandContour_rank,PoolArea,LotConfig_rank,ScreenPorch,SaleCondition_rank,RoofStyle_rank,BedroomAbvGr,BsmtUnfSF,BsmtFullBath,LotArea,LotShape_rank,HalfBath,OpenPorchSF,2ndFlrSF,WoodDeckSF,BsmtFinSF1,Fireplaces,YearRemodAdd,YearBuilt,TotRmsAbvGrd,FullBath,1stFlrSF,TotalBsmtSF,GarageArea,GarageCars,GrLivArea,OverallQual
0,2,1,1,1,0,1,0,1,1,3,150,1,8450,1,1,61,854,0,706,0,2003,2003,8,2,856,856,548,2,1710,7
1,5,1,1,1,0,2,0,1,1,3,284,0,9600,1,0,0,0,298,978,1,1976,1976,6,2,1262,1262,460,2,1262,6
2,9,1,1,1,0,1,0,1,1,3,434,1,11250,2,1,42,866,0,486,1,2002,2001,6,2,920,920,608,2,1786,7
3,2,1,1,1,0,3,0,2,1,3,540,1,9550,2,0,35,756,0,216,1,1970,1915,7,1,961,756,642,3,1717,7
4,12,1,1,1,0,2,0,1,1,4,490,1,14260,2,1,84,1053,192,655,1,2000,2000,9,2,1145,1145,836,3,2198,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,8,1,1,1,0,1,0,1,1,3,953,0,7917,1,1,40,694,0,0,1,2000,1999,7,2,953,953,460,2,1647,6
1456,2,1,1,1,0,1,0,1,1,3,589,1,13175,1,0,0,0,349,790,2,1988,1978,7,2,2073,1542,500,2,2073,6
1457,5,1,1,1,0,1,0,1,1,4,877,0,9042,1,0,60,1152,0,275,2,2006,1941,9,2,1188,1152,252,1,2340,7
1458,4,1,1,1,0,1,0,1,2,2,0,1,9717,1,0,0,0,366,49,0,1996,1950,5,1,1078,1078,240,1,1078,5


In [68]:
y

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [69]:
# train_test_split을 해준다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [70]:
# normalize를 하는 방법은 다양하지만, 쉽게 많이 접근하는 것이 최댓값과 최솟값으로 scaling하는 것이다.
# 이 과정을 거치면 column의 범위가 0에서부터 1이 된다.

from sklearn.preprocessing import MinMaxScaler

In [71]:
# column의 min 값과 column의 max 값을 구하면 column 전체를 의미한다.
# x에 min이 들어가면 결과가 0이 되고, max가 들어가면 결과가 1이 된다.
# 따라서 column의 범위가 0에서 1이 된다.
# x = (x - min(column)) / (max(column) - min(column))

# column 별로 min과 max가 다르기 때문에 쉽게 하기 위해 사이킷런에 준비된 클래스를 사용할 것이다.
min_max_scaler = MinMaxScaler()

In [73]:
# min_max_scaler.fit()을 하게 되면 x_train의 column별로 최대 최소값을 저장한다.
min_max_scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [74]:
# transform()을 사용하면 특정 행렬의 scale을 fitting한 최대최소 값에 따라 바꾸게 된다.
scaled_X_train = min_max_scaler.transform(X_train)

In [75]:
# X_test도 똑같이 scaling을 해준다.
scaled_X_test = min_max_scaler.transform(X_test)

#### y값 scaling

In [76]:
y_min_max_scaler = MinMaxScaler()

In [77]:
# 위에서는 min_max_scaler에 X_train을 바로 넣어줬다.
# 하지만 이게 작동할 수 있었던 이유는 X_train의 shape은 1314x30의 행렬인데,
# y_train의 shape은 그냥 1314이다.
X_train.shape

(1314, 30)

In [78]:
# 0번째 axis는 있는데, 1번째 axis가 없다.
y_train.shape

(1314,)

In [79]:
# 이러면 api의 설계상 작동하지 않기 때문에 shape을 1314x1로 reshape을 해줘야 fitting이 된다.
# 따라서 reshape을 해준 상태로 fit에 넣어줘야 한다.
y_min_max_scaler.fit(np.array(y_train).reshape(-1, 1))

MinMaxScaler(copy=True, feature_range=(0, 1))

In [80]:
# y_train scaling
scaled_y_train = y_min_max_scaler.transform(np.array(y_train).reshape(-1, 1))

In [81]:
# y_test scaling
scaled_y_test = y_min_max_scaler.transform(np.array(y_test).reshape(-1, 1))

In [82]:
# scaling된 X_train과 X_test의 shape
scaled_X_train.shape, scaled_X_test.shape

((1314, 30), (146, 30))

In [83]:
# scaling된 y_train과 y_test의 shape
scaled_y_train.shape, scaled_y_test.shape

((1314, 1), (146, 1))

### MLP model architecture

In [84]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [91]:
# 간단한 모델을 우선 한번 만들어보자.
# Input에 input으로 받는 feature size를 넣어준다.
# output size는 1로 해주고, regularizer는 overfitting을 방지하기 위해 사용한다.
# regularizer로 l1을 사용했기 때문에 Lasso와 거의 동일한 형태가 된다.
# lambda의 값은 0.01이다. (keras의 기본값도 0.01)
model = keras.Sequential(
    [
        keras.Input(shape=scaled_X_train.shape[-1]),
        layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l1(0.01))
    ]
)

In [92]:
# parameter 수가 31이 되는 이유는 weight가 30이고 bias가 1이라서 31이 된다.
# y = Xw + b
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1)                 31        
Total params: 31
Trainable params: 31
Non-trainable params: 0
_________________________________________________________________


In [93]:
# loss는 mean squared error로 주고, optimizer는 adam으로 함.
model.compile(loss="mse", optimizer="adam")

In [94]:
# fit을 하면 loss가 점점 줄어든다.
model.fit(scaled_X_train, scaled_y_train, batch_size=1, epochs=100)

Epoch 1/100
1314/1314 [==============================] - 1s 816us/step - loss: 0.0757
Epoch 2/100
1314/1314 [==============================] - 1s 833us/step - loss: 0.0184
Epoch 3/100
1314/1314 [==============================] - 1s 896us/step - loss: 0.0107
Epoch 4/100
1314/1314 [==============================] - 1s 838us/step - loss: 0.0096
Epoch 5/100
1314/1314 [==============================] - 1s 900us/step - loss: 0.0094
Epoch 6/100
1314/1314 [==============================] - 1s 835us/step - loss: 0.0093
Epoch 7/100
1314/1314 [==============================] - 1s 851us/step - loss: 0.0093
Epoch 8/100
1314/1314 [==============================] - 1s 865us/step - loss: 0.0093
Epoch 9/100
1314/1314 [==============================] - 1s 850us/step - loss: 0.0092
Epoch 10/100
1314/1314 [==============================] - 1s 861us/step - loss: 0.0093
Epoch 11/100
1314/1314 [==============================] - 1s 852us/step - loss: 0.0092
Epoch 12/100
1314/1314 [============================

In [95]:
# predict를 할땐 test set을 넣어야한다.
# 그런데 학습할 때 scale된 값을 넣었으니, predict할때도 scale된 test set을 넣어야 한다.
scaled_pred = model.predict(scaled_X_test)

In [96]:
# 그럼 scale이 변형된 predict값이 나온다.
# 이걸 우리가 평가를 하려면 원래 scale로 되돌려야한다.
# 이때 사용하는 것이 inverse_transform이다.
pred_y = y_min_max_scaler.inverse_transform(scaled_pred)

In [97]:
print_evaluate(y_test, pred_y)

MAE:  37847.03403253425
MSE:  4155276741.7544723
RMSE:  64461.436082005435
R2 Square:  0.5452073375802817


### Add layers

In [109]:
# 방금까지 굉장히 간단한 모델을 학습시켰다.
# 여기서 더 할 수 있는 것은 깊이를 깊게 하거나 (hidden layer를 늘리겠다는 뜻)
# feature의 size를 늘릴 수도 있다. (weight들을 많이 쓰겠다는 뜻)

# 우리가 할 것은 hidden layer를 넣는 것이다.

# activation을 사용하지 않으면 그냥 선형 모델이 되기 때문에
# 조금 더 복잡한 문제를 풀기 위해 non-linearity를 주려고 activation을 사용했다.

model = keras.Sequential(
    [
        keras.Input(shape=scaled_X_train.shape[-1]),
        layers.Dense(96, activation='relu'),
        layers.Dense(48, activation='relu'),
        layers.Dense(1)
    ]
)

In [110]:
# 학습 데이터가 어마어마하게 커짐
# 31 -> 7681
# input에 비해 너무 많은 양이라 overfitting이 발생할 가능성이 높다.
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 96)                2976      
_________________________________________________________________
dense_7 (Dense)              (None, 48)                4656      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 49        
Total params: 7,681
Trainable params: 7,681
Non-trainable params: 0
_________________________________________________________________


In [111]:
model.compile(loss="mse", optimizer="adam")

In [112]:
# loss가 계속 떨어진다는 것은 어떤 시점 이후로 overfitting이 발생하고 있다라고 생각하면 된다.
# 이럴 땐 validation set을 설정하여 overfitting이 되고 있는지 아닌지 체크해줄 필요가 있다.
# 뒤에 validation_split을 넣어주면 overfit이 나는지 아닌지 확인해준다.

# 이때 validation loss가 어느 시점 이후로 증가하거나,
# loss와 validation loss가 차이가 많이 나면
# overfitting이 진행되고 있는 것이므로 학습을 중단시켜야 한다.

model.fit(scaled_X_train, scaled_y_train, batch_size=2, epochs=100, validation_split=0.05)

Epoch 1/100
624/624 [==============================] - 1s 1ms/step - loss: 0.0068 - val_loss: 0.0034
Epoch 2/100
624/624 [==============================] - 1s 1ms/step - loss: 0.0035 - val_loss: 0.0029
Epoch 3/100
624/624 [==============================] - 1s 1ms/step - loss: 0.0031 - val_loss: 0.0025
Epoch 4/100
624/624 [==============================] - 1s 1ms/step - loss: 0.0029 - val_loss: 0.0028
Epoch 5/100
624/624 [==============================] - 1s 1ms/step - loss: 0.0024 - val_loss: 0.0033
Epoch 6/100
624/624 [==============================] - 1s 1ms/step - loss: 0.0022 - val_loss: 0.0025
Epoch 7/100
624/624 [==============================] - 1s 1ms/step - loss: 0.0022 - val_loss: 0.0023
Epoch 8/100
624/624 [==============================] - 1s 1ms/step - loss: 0.0020 - val_loss: 0.0018
Epoch 9/100
624/624 [==============================] - 1s 1ms/step - loss: 0.0018 - val_loss: 0.0026
Epoch 10/100
624/624 [==============================] - 1s 1ms/step - loss: 0.0021 - val_lo

In [113]:
scaled_pred = model.predict(scaled_X_test)
pred_y = y_min_max_scaler.inverse_transform(scaled_pred)

In [114]:
print_evaluate(y_test, pred_y)

MAE:  20203.42615582192
MSE:  864479580.8491796
RMSE:  29402.033617577876
R2 Square:  0.9053832043889626


### Early-stopping

In [122]:
# 데이터의 양이 많을 땐 우리가 직접 보고 학습을 중단시킬 수 없기 때문에
# 알아서 어느 시점에 꺼지도록 하는 방법이 early-stopping 기능이다.
model = keras.Sequential(
    [
        keras.Input(shape=scaled_X_train.shape[-1]),
        layers.Dense(96, activation='relu'),
        layers.Dense(48, activation='relu'),
        layers.Dense(1)
    ]
)
model.compile(loss="mse", optimizer="adam")

In [123]:
# patience 수 만큼 학습을 하면서 validation loss가 줄지 않으면 학습을 중단시킨다. 
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)

In [124]:
model.fit(scaled_X_train, scaled_y_train, batch_size=2, epochs=150, validation_split=0.1, callbacks=[callback])

Epoch 1/150
591/591 [==============================] - 1s 1ms/step - loss: 0.0042 - val_loss: 0.0022
Epoch 2/150
591/591 [==============================] - 1s 1ms/step - loss: 0.0034 - val_loss: 0.0021
Epoch 3/150
591/591 [==============================] - 1s 1ms/step - loss: 0.0027 - val_loss: 0.0015
Epoch 4/150
591/591 [==============================] - 1s 1ms/step - loss: 0.0027 - val_loss: 0.0014
Epoch 5/150
591/591 [==============================] - 1s 1ms/step - loss: 0.0025 - val_loss: 0.0015
Epoch 6/150
591/591 [==============================] - 1s 1ms/step - loss: 0.0020 - val_loss: 0.0045
Epoch 7/150
591/591 [==============================] - 1s 1ms/step - loss: 0.0021 - val_loss: 0.0014
Epoch 8/150
591/591 [==============================] - 1s 1ms/step - loss: 0.0018 - val_loss: 0.0018
Epoch 9/150
591/591 [==============================] - 1s 1ms/step - loss: 0.0017 - val_loss: 9.4031e-04
Epoch 10/150
591/591 [==============================] - 1s 1ms/step - loss: 0.0023 - va